In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import norm
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.optimize import curve_fit
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt;
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error as mse
import itertools
plt.rcdefaults()

In [1]:
s1 = set((0,1, 2, 3, 6, 12))
triples = list(itertools.product(s1,s1,s1))

def predict_arima(X, months=24, colname="y"):
    preds_ = []
    mse_ = np.inf
    for i in triples:
        print("i=", i)
        try:
            sar = SARIMAX(X, order=i).fit()
            preds = sar.predict(start = 0, end= len(X) -1 + months, dynamic=False)
            c = n + "_" + str(colname) + ",SARIMAX("+str(i)+f"), residuals({model_type},{period})"
            print("len_preds", len(preds.fillna(0)))
            print("len_df", len(differenced_residuals_df_preds))
            differenced_residuals_df_preds[c] = preds.fillna(0)
        except Exception as er:
            print("error", er)
            continue
    return preds_


for col in differenced_residuals_df.columns:
    predict_arima(differenced_residuals_df[col], 24, col)
    
    

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LinearRegression, Ridge, PoissonRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
models = {
    "DecisionTreeRegressor": {"model": DecisionTreeRegressor, "params":{
        "max_depth":[1,2]
                              }},
    "Penelized": {"alpha": [1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]},
    "LinearRegression":{"model":Pipeline([("scl", StandardScaler()), ("reg", LinearRegression())]), "params":{
    }},
    "PoissonRegression":{"model":Pipeline([("scl", StandardScaler()), ("reg", PoissonRegressor())]), 
                         "params":{"reg__alpha": [0.01, 0.1, 1, 10]}},
    "PoissonRegression":{"model":Pipeline([("scl", StandardScaler()), ("reg", Ridge(random_state=42))]), 
                         "params":{"reg__alpha": [0.01, 0.1, 1, 10]}},
    "SVR":{"model":Pipeline([("scl", StandardScaler()), ("reg", SVR())]), 
                        "params":{"reg__C": [0.01, 0.1, 1, 10],
                            "reg__kernel": ["rbf", "linear"]}},
}

In [3]:
def tune_model_parameters(model, parameters_dict, X_set, y_set, verbose=False):
  cv = TimeSeriesSplit(n_splits=2, test_size=int(len(y_set)*0.15))
  clf = GridSearchCV(model, parameters_dict, scoring='neg_root_mean_squared_error', cv=cv, verbose=verbose)
  clf.fit(X_set, y_set)
  if verbose:
    print("Current best model: ", clf.best_estimator_)
    print("Current best model: ", clf.best_score_)
  return clf.best_estimator_, clf.best_score_

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [47]:
n="df_uah1"
n1="uah1"
model_type = "multiplicative"
period = 12

t1_residuals = pd.read_csv(f"/Users/Marta/Desktop/diploma/best/{model_type}_{period}_{n1}/best_{n}_{model_type}_{period}_decomposed_RESIDUALS_predicted_T1.csv")
t2_residuals = pd.read_csv(f"/Users/Marta/Desktop/diploma/best/{model_type}_{period}_{n1}/best_{n}_{model_type}_{period}_decomposed_RESIDUALS_predicted_T2.csv")
total1_residuals = pd.read_csv(f"/Users/Marta/Desktop/diploma/best/{model_type}_{period}_{n1}/best_{n}_{model_type}_{period}_decomposed_RESIDUALS_predicted_Total1.csv")


t1s = [col for col in t1_residuals.columns if "T1" in col and len(col)>8 and "FB" not in col]
t2s = [col for col in t2_residuals.columns if "T2" in col and len(col)>8 and "FB" not in col]
total1s = [col for col in total1_residuals.columns if "Total 1" in col and len(col)>8 and "FB" not in col]


In [49]:
all_residuals = pd.concat([t1_residuals, t2_residuals, total1_residuals], axis=1)
t1s_regressed = pd.DataFrame()
t2s_regressed = pd.DataFrame()
total1s_regressed = pd.DataFrame()

In [50]:
s1 = set((0,1, 2, 3, 6, 12))
triples = list(itertools.product(s1,s1,s1))

for triple in triples:
    for col1 in t1s:
        for col2 in t2s:
            for col3 in total1s:
                if str(triple) not in col1 or str(triple) not in col2 or str(triple) not in col3:
                    continue
                print(col1, col2, col3)
                X = all_residuals[[col1, col2, col3]]
                Xs = [all_residuals[[col1, col2]], all_residuals[[col1, col3]], all_residuals[[col3, col2]]]
                ys = [all_residuals[col3], all_residuals[col2], all_residuals[col1]]
                preds_dfs = [total1s_regressed, t2s_regressed, t1s_regressed]

                for i in range(0, 3):
                    best_est = None
                    best_score = -np.inf
                    print(Xs[i])
                    for model in models:
                        print(model)
                        try:
                            est, score = tune_model_parameters(models[model]["model"](), models[model]["params"], Xs[i], ys[i], False)
                        except TypeError:
                            est, score = tune_model_parameters(models[model]["model"], models[model]["params"], Xs[i], ys[i], False)

                        prediction = est.predict(Xs[i])
                        c = "SARIMAX("+str(triple)+str(model)+", regressed"
                        preds_dfs[i][c] = prediction
                        plt.plot(ys[i], color="black")
                        plt.plot(Xs[i].index, prediction, color="blue")
                        if best_score < score:
                            best_est = est
                            best_score = score
